# Inference Pipeline

In [2]:
from pathlib import Path
import json
import numpy as np
from yolov7.utils.data_preparation import copy_split, copytree_with_suffix
import os
import shutil

## Split Training and Test Data on KITTI

In [4]:
# Paths to image and label data (for loading the KITTI dataset within Kaggle)
base_dir = Path.cwd().parent.parent / "input"
print(f"Base Directory : {base_dir}")
img_path = base_dir / "images" / "data_object_image_2" / "training" / "image_2"
label_path = base_dir / "labels"

# Loading the classes present in the dataset
with open(base_dir / "classes.json", "r") as f:
    classes = json.load(f)
print(f"Classes : {classes}")

# Sorting images and labels to ensure alignment
ims = sorted(img_path.glob("*"))
labels = sorted(label_path.glob("*"))
pairs = list(zip(ims, labels))

# Dataset shuffle for randomized train/test split
seed = 42  # For reproducibility
random_state = np.random.RandomState(seed)
random_state.shuffle(pairs)

# Calculating the test size (10%)
test_size = int(0.1 * len(pairs))
splits = {}

# Creating 3 distinct splits
for i in range(3):
    # Select the test set for this split
    test_set = pairs[i * test_size : (i + 1) * test_size]
    # Select the training set (remaining data)
    train_set = pairs[: i * test_size] + pairs[(i + 1) * test_size :]
    splits[f"split{i + 1}"] = {"train": train_set, "test": test_set}

# Verifying sizes of each split
for key, value in splits.items():
    train_size = len(value["train"])
    test_size = len(value["test"])
    print(f"{key} - Train Size: {train_size}, Test Size: {test_size}")
    assert train_size + test_size == len(
        pairs
    ), "Train and test sizes do not add up to total pairs"

# Verifying distinct test sets
for i in range(3):
    for j in range(i + 1, 3):
        assert not set(splits[f"split{i + 1}"]["test"]).intersection(
            set(splits[f"split{j + 1}"]["test"])
        ), f"Test sets for split{i + 1} and split{j + 1} overlap"


Base Directory : D:\dev-workspace\IRO\input
Classes : {'Car': 0, 'Pedestrian': 1, 'Van': 2, 'Cyclist': 3, 'Truck': 4, 'Misc': 5, 'Tram': 6, 'Person_sitting': 7}
split1 - Train Size: 6733, Test Size: 748
split2 - Train Size: 6733, Test Size: 748
split3 - Train Size: 6733, Test Size: 748


## Inference Using YOLOv7 Trained on KITTI Dataset

In [15]:
test_path = Path('../test').resolve()

if test_path.exists():
    shutil.rmtree(test_path)

In [16]:
# Get test data split index and path
# test_split = splits['split1']['test']
test_split = splits['split2']['test']
# test_split = splits['split3']['test']

# Copy test dataset to a designated folder
copy_split(test_split, test_path)

100%|██████████| 748/748 [00:09<00:00, 82.22it/s]


In [12]:
%env WANDB_DISABLED=True

# Run inference script
!python ../detect.py --weights ../yolov7-tiny-kitti.pt --conf 0.25 --img-size 640 --source ../test --save-txt --save-conf


env: WANDB_DISABLED=True
Namespace(weights=['../yolov7-tiny-kitti.pt'], source='../test', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 Car, 5 Pedestrians, Done. (6.2ms) Inference, (83.1ms) NMS
 The image with the result is saved in: runs\detect\exp\000015.png
1 Car, 1 Truck, Done. (10.5ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\000026.png
1 Car, 1 Misc, Done. (10.1ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\000029.png
3 Cars, Done. (7.3ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\000033.png
2 Cars, Done. (7.0ms) Inference, (0.0ms) NMS
 The image with the 

YOLOR  62efa3d torch 2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4095.5MB)

D:\dev-workspace\IRO\yolov7\models\experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experim

## Inference Using YOLOv7 Pretrained on COCO Dataset

In [17]:
%env WANDB_DISABLED=True

# Run inference script
!python ../detect.py --weights ../yolov7-tiny.pt --conf 0.25 --img-size 640 --source ../test --save-txt --save-conf


env: WANDB_DISABLED=True
Namespace(weights=['../yolov7-tiny.pt'], source='../test', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 person, 3 bicycles, Done. (15.1ms) Inference, (105.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\000000.png
8 cars, Done. (6.2ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\000008.png
1 car, Done. (4.6ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\000014.png
1 car, 2 traffic lights, 1 fire hydrant, Done. (11.4ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\000017.png
1 car, 2 trains, Done. (5.8ms) Inference, (2.5ms) NMS
 The ima

YOLOR  919876e torch 2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4095.5MB)

D:\dev-workspace\IRO\yolov7\models\experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experim

2 persons, 5 bicycles, Done. (20.2ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\003997.png
2 cars, Done. (20.2ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\004015.png
5 cars, 1 motorcycle, Done. (20.2ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\004019.png
7 persons, 1 backpack, 2 handbags, Done. (23.7ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\004039.png
2 cars, Done. (15.2ms) Inference, (5.1ms) NMS
 The image with the result is saved in: runs\detect\exp5\004046.png
10 persons, 3 bicycles, 1 stop sign, 1 chair, 1 potted plant, Done. (20.3ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\004055.png
2 persons, 1 car, Done. (11.1ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp5\004059.png
1 car, Done. (15.2ms) Inference, (0.0ms) NMS
 The image with the result is saved i

## Inference on Gazebo Data

### Copy All Data Folders

In [71]:
data_path = Path("../../gazebo_data")
data_folders = os.listdir(data_path)

test_path = Path('../test').resolve()
if test_path.exists():
    shutil.rmtree(test_path)

In [72]:
# for data_folder in data_folders:
#     copytree_with_suffix(data_path / data_folder / 'images', test_path, f'_{data_folder}')

In [73]:
# Weather condition
# data_folders = ["asphalt_clear", "asphalt_cloudy", "gray_clear", "gray_cloudy", "gray_night", "gray_sunset"]
# env = "fog"     # "", "rain", "fog"
# for data_folder in data_folders:
#     copytree_with_suffix(data_path / f"{data_folder}_{env}" / 'images', test_path, f'_{data_folder}_{env}')
#     # copytree_with_suffix(data_path / data_folder / 'labels', test_path, f'_{data_folder}')

In [74]:
# Time of Day
tod = "night"   # "clear", "cloudy", "sunset", "night"
data_folders = os.listdir(data_path)
for data_folder in data_folders:
    if tod in data_folder:
        copytree_with_suffix(data_path / data_folder / 'images', test_path, f'_{data_folder}')

### Inference Mutliple Folders

In [77]:
%env WANDB_DISABLED=True

# Run inference script
!python ../detect.py --weights ../yolov7-tiny-kitti.pt --conf 0.25 --img-size 640 --source ../test --save-txt --save-conf --nosave


env: WANDB_DISABLED=True
Namespace(weights=['../yolov7-tiny-kitti.pt'], source='../test', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=True, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 Car, 2 Pedestrians, Done. (3.7ms) Inference, (69.8ms) NMS
1 Car, 2 Pedestrians, Done. (8.2ms) Inference, (2.1ms) NMS
1 Car, 2 Pedestrians, Done. (10.1ms) Inference, (0.0ms) NMS
1 Car, 1 Pedestrian, Done. (10.2ms) Inference, (0.0ms) NMS
1 Car, 1 Van, 1 Misc, Done. (7.3ms) Inference, (2.9ms) NMS
1 Pedestrian, Done. (6.2ms) Inference, (1.0ms) NMS
1 Car, 1 Pedestrian, Done. (3.2ms) Inference, (0.0ms) NMS
1 Car, 1 Van, Done. (0.0ms) Inference, (7.3ms) NMS
Done. (10.3ms) Inference, (0.0ms) NMS
2 Cars, 1 Pedestrian, 1 Van, Done. (7.5ms) Inferenc

YOLOR  d5bbc42 torch 2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4095.5MB)

D:\dev-workspace\IRO\yolov7\models\experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experim

### Inference Single Data Folder

In [ ]:
# %env WANDB_DISABLED=True
# 
# # Run inference script
# !python ../detect.py --weights ../yolov7-tiny-kitti.pt --conf 0.25 --img-size 640 --source ../../gazebo_data/asphalt_clear/images --save-txt --save-conf --project ../../gazebo_data/asphalt_clear --name detections
